<a href="https://colab.research.google.com/github/asmaamuhamed20/water-segmentation/blob/main/water_segmentation_(pretrain_model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [152]:
import os
import numpy as np
import tifffile
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, concatenate, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
import tensorflow.keras.backend as K

In [ ]:
image_folder = '/content/drive/MyDrive/satalite_data2/images'
mask_folder = '/content/drive/MyDrive/satalite_data2/labels'

In [153]:
def load_image(image_path):
    path = image_path.numpy().decode('utf-8')
    img = tifffile.imread(path).astype(np.float32) / 65535.0
    return img

def load_image_tf(image_path):
    img = tf.py_function(load_image, [image_path], tf.float32)
    img.set_shape([128, 128, 12])
    return img

def load_mask(mask_path):
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)
    mask = tf.cast(mask > 127, tf.float32)
    mask.set_shape([128, 128, 1])
    return mask

def load_and_preprocess(image_path, mask_path):
    image = load_image_tf(image_path)
    mask = load_mask(mask_path)
    return image, mask


In [154]:
image_paths_ds = tf.data.Dataset.from_tensor_slices(valid_image_paths)
mask_paths_ds = tf.data.Dataset.from_tensor_slices(valid_mask_paths)
dataset = tf.data.Dataset.zip((image_paths_ds, mask_paths_ds))
dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=100).batch(16).prefetch(tf.data.AUTOTUNE)


In [155]:
def iou_metric(y_true, y_pred, smooth=1e-6):
    y_pred = K.cast(y_pred > 0.5, 'float32')
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred) - intersection
    return (intersection + smooth) / (union + smooth)


In [ ]:
import matplotlib.pyplot as plt

def visualize_sample(dataset, index=0):
    for image, mask in dataset.skip(index).take(1):
        image_np = image.numpy()
        mask_np = mask.numpy().squeeze()

        fig, axes = plt.subplots(3, 4, figsize=(12, 9))
        for i in range(12):
            ax = axes[i//4, i%4]
            ax.imshow(image_np[:, :, i], cmap='gray')
            ax.set_title(f'Band {i+1}')
            ax.axis('off')

        plt.tight_layout()
        plt.show()

        plt.figure(figsize=(4, 4))
        plt.imshow(mask_np, cmap='gray')
        plt.title("Mask")
        plt.axis('off')
        plt.show()


visualize_sample(dataset)

In [156]:
def unet_resnet50(input_shape=(128, 128, 12)):
    base_model = ResNet50(include_top=False, weights=None, input_shape=input_shape)

    for layer in base_model.layers:
        layer.trainable = True

    skip_names = ["conv1_relu", "conv2_block3_out", "conv3_block4_out", "conv4_block6_out"]
    skips = [base_model.get_layer(name).output for name in skip_names]

    x = base_model.output

    def conv_block(x, filters):
        x = Conv2D(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        return x

    x = UpSampling2D()(x)
    x = concatenate([x, skips[-1]])
    x = conv_block(x, 512)

    x = UpSampling2D()(x)
    x = concatenate([x, skips[-2]])
    x = conv_block(x, 256)

    x = UpSampling2D()(x)
    x = concatenate([x, skips[-3]])
    x = conv_block(x, 128)

    x = UpSampling2D()(x)
    x = concatenate([x, skips[-4]])
    x = conv_block(x, 64)

    x = UpSampling2D()(x)
    x = conv_block(x, 32)

    outputs = Conv2D(1, 1, activation="sigmoid")(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    return model

In [157]:
# ========== تدريب الموديل ==========
model = unet_resnet50()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', iou_metric])
model.fit(dataset, epochs=20)

Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 317s 13s/step - accuracy: 0.8494 - iou_metric: 0.0120 - loss: 0.4673
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 250s 12s/step - accuracy: 0.9986 - iou_metric: 0.3127 - loss: 0.1832
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 254s 13s/step - accuracy: 0.9994 - iou_metric: 0.6301 - loss: 0.1395
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 250s 12s/step - accuracy: 0.9999 - iou_metric: 0.6873 - loss: 0.1038
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 263s 12s/step - accuracy: 0.9999 - iou_metric: 0.8530 - loss: 0.0792
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 251s 12s/step - accuracy: 0.9997 - iou_metric: 0.4907 - loss: 0.0635
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 264s 13s/step - accuracy: 1.0000 - iou_metric: 0.7281 - loss: 0.0530
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 251s 12s/step - accuracy: 1.0000 - iou_metric: 1.0000 - loss: 0.0456
Epoch 9/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 264s 13s/step - accuracy: 1.0000 - iou_metric: 0.9701 - loss: 0.0391
Epoch 10/20
20/20 ━━━━━━━━━━━━━━━━━━━